In [1]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes


In [ ]:
#case control swimming speed, ran this in screen so didn't have to wait for what's already running in the nb 
#- -do HWE1, -minHWEpval 0.05 / total number of loci(7596774)=
!angsd -yBin angsd/case_control_pheno.ybin -doAsso 1 -doPost 1 -cov angsd/pc1.cov -GL 1 -doMajorMinor 1 -doMaf 1 -nThreads 20 -minQ 20 -minMapQ 20 -SNP_pval 1e-6 -minInd 16 -minMAF 0.01  -fai /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta.fai -out angsd/case_control_assoc_out -bam bamlist.list 

	-> angsd version: 0.918 (htslib: 1.5) build(Jul  5 2017 14:48:45)
	-> SNP-filter using a pvalue: 1.000000e-06 correspond to 23.928127 likelihood units
	-> Parsing 159 number of samples 

	-> Allocated ~ 10 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 20 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 30 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 40 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 50 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 60 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 70 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 80 million nodes to the nodepool, this is not an estimate of the memory usage
	-> Printing at chr: CM009708.1 pos:76549 chunknumber

In [ ]:
!angsd -beagle angsd/appc_angsd_probs.beagle.gz -yQuant angsd/case_control_pheno.ybin -fai /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta.fai -doAsso 5 -cov angsd/output.pcangsd.cov -doMaf 4 -nThreads 20  -out angsd/case_control_assoc_out2 

In [23]:
#case control swimming speed NULL 
!angsd -yBin angsd/case_control_pheno_NULL.ybin -doAsso 1 -doPost 1 -cov angsd/pc1.cov -GL 1 -doMajorMinor 1 -doMaf 1 -nThreads 20 -minQ 20 -minMapQ 20 -SNP_pval 1e-6 -minInd 16 -minMAF 0.01  -fai /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta.fai -out angsd/case_control_assoc_out_NULL -bam bamlist.list 

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> SNP-filter using a pvalue: 1.000000e-06 correspond to 23.928127 likelihood units
[bammer_main] 159 samples in 159 input files
	-> Parsing 159 number of samples 

	-> Allocated ~ 10 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 20 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 30 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 40 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 50 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 60 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 70 million nodes to the nodepool, this is not an estimate of the memory usage

	-> Allocated ~ 80 million nodes to the nodepool, this is not an estimate of the memory usage

	-> All

In [ ]:
#once I get the probabilities as output from do post, then use those for the hybrid quant model so I can get effect size estimates. in the mean time use qctools to look at stratification

In [26]:
!pwd

/local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes


In [31]:
#quant SL
!angsd -beagle angsd/appc_angsd_probs.beagle.gz -yQuant angsd/SL_pheno.ybin -fai /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta.fai -doAsso 5 -cov angsd/output.pcangsd.cov -doMaf 4 -nThreads 40  -out angsd/SL_quant_assoc_out 

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000272.1 pos:14094 chunknumber 135200 contains 3 sitestesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:39
	-> Printing at chr: PYAM01000096.1 pos:10177 chunknumber 135300 contains 50 sites	-> Done waiting for threads
	-> Output filenames:
		->"angsd/SL_quant_assoc_out.arg"
		->"angsd/SL_quant_assoc_out.mafs.gz"
		->"angsd/SL_quant_assoc_out.lrt0.gz"
	-> Tue Mar  1 11:02:24 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 6758560
	-> Number of sites retained after filtering: 6758560 
	[ALL done] cpu-time used =  122802.43 sec
	[ALL done] walltime used =  3143.00 sec


In [33]:
#quant Ucrit
!angsd -beagle angsd/appc_angsd_probs.beagle.gz -yQuant angsd/Ucrit_pheno.ybin -fai /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/genotypes/reference.fasta.fai -doAsso 5 -minHigh 20 -cov angsd/output.pcangsd.cov -doMaf 4 -nThreads 40  -out angsd/Ucrit_quant_assoc_out 

	-> angsd version: 0.933 (htslib: 1.9) build(May  6 2020 21:25:11)
	-> Printing at chr: PYAM01000272.1 pos:14094 chunknumber 135200 contains 3 sitestesDone reading beagle
	-> Done reading data waiting for calculations to finish
	-> Waiting for nThreads:39
	-> Printing at chr: PYAM01000096.1 pos:10177 chunknumber 135300 contains 50 sites	-> Done waiting for threads
	-> Output filenames:
		->"angsd/Ucrit_quant_assoc_out.arg"
		->"angsd/Ucrit_quant_assoc_out.mafs.gz"
		->"angsd/Ucrit_quant_assoc_out.lrt0.gz"
	-> Tue Mar  1 14:04:08 2022
	-> Arguments and parameters for all analysis are located in .arg file
	-> Total number of sites analyzed: 6758560
	-> Number of sites retained after filtering: 6758560 
	[ALL done] cpu-time used =  128992.64 sec
	[ALL done] walltime used =  3293.00 sec
